In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import plotly
import plotly.plotly as py
import plotly.tools as tls
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt

# Function to create Variables per file

In [ ]:
def season_to_np (df):
    # Team Strength
    # Through SPI 
    
    # Cumulative values
    away_items = ['AC', 'AF', 'AR', 'AS', 'AST', 'AY','FTAG']
    home_items = ['HC', 'HF', 'HR', 'HS', 'HST', 'HY','FTHG']
    for item in home_items:
        df[("C_"+str(item))] = (df.groupby('HomeTeam')[item].transform(pd.Series.cumsum))-df[item]
    for item in away_items:
        df[("C_"+str(item))] = (df.groupby('AwayTeam')[item].transform(pd.Series.cumsum))-df[item]
    
    # Map result to points
    df['FTHP'] = df['FTR'].map({'H': 3, 'D': 1, "A": 0})
    df['FTAP'] = df['FTR'].map({'H': 0, 'D': 1, "A": 3})
    
    # Cumulative points
    df[("C_FTHP")] = (df.groupby('HomeTeam')['FTHP'].transform(pd.Series.cumsum))-df['FTHP']
    df[("C_FTAP")] = (df.groupby('AwayTeam')['FTAP'].transform(pd.Series.cumsum))-df['FTAP'] 
    
    # Form
    # Form Home Team for Home matches
    df['F_HT'] = df.groupby('HomeTeam', as_index=False)['FTHP'].rolling(5).sum().reset_index(0, drop=True)-df['FTHP']
    # Form Away Team for Away matches
    df['F_AT'] = df.groupby('AwayTeam', as_index=False)['FTAP'].rolling(5).sum().reset_index(0, drop=True)-df['FTAP']

    # Difference between matches
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')
    df['D_HM'] = df.groupby(['HomeTeam'])['Date'].diff().fillna(0).dt.days#.astype(int)
    df['D_AM'] = df.groupby('AwayTeam')['Date'].diff().fillna(0).dt.days#.astype(int)
    
    return df

# Find all file names

In [ ]:
leagues = [" Premier League", " Championship", " League 1", " League 2"]
seasons = ["Season "+str(i)+" "+str(i+1) for i in range(2006,2018,1)]
all_files = []

for season in seasons:
    for league in leagues:
        all_files.append("files/"+str(season)+str(league)+".csv")
#print(len(all_files))
#print(all_files[10])


# Add Season & Div column to all files 

In [ ]:
# add column with Season
for file in all_files:
    df = pd.read_csv(file)
    df['Season']=str(file)
    df.to_csv(file, index=False)


In [ ]:
# add column with Division
for file in all_files:
    for league in leagues:
        if league in file:
            df = pd.read_csv(file)
            df['Div']=league
            df.to_csv(file, index=False)


# Engineer, concat & Save to csv

In [ ]:
li = []

for filename in all_files[1:25]:
    df = season_to_np(pd.read_csv(filename, index_col=0))#header=0
    li.append(df)
    # till 25 works well then mismatch
    
for filename in all_files[25:40]:
    df = season_to_np(pd.read_csv(filename, index_col=0, skipfooter=2, error_bad_lines=False, engine='python'))#header=0
    li.append(df) 
    #Mismatched solved by skipfooter and error_bad_lines
    
for filename in all_files[40:]:
    df = season_to_np(pd.read_csv(filename, index_col=0))#header=0
    li.append(df)
    #works again from 40:


df = pd.concat(li, axis=0, ignore_index=True, sort=True)
df.to_csv("all_files.csv", index=False)


# Add column with team strength / SPI


In [ ]:
data = pd.read_csv("all_files.csv")
print(data.shape)
spi = pd.read_csv("spi_england1", index_col='name', usecols=['name', 'spi'])
data = data.join(spi, on = 'HomeTeam', rsuffix='_H')
data = data.join(spi, on = 'AwayTeam', rsuffix='_A')

data.to_csv("all_files_spi.csv", index=False)
print(data.shape)
#data[("Form_FTHP")] = data.groupby('HomeTeam')['FTHP'].rolling(7).sum()
#data[('Form_FTHP')] 

# Factorize & select required columns

In [4]:
df = pd.read_csv("all_files_spi.csv")
df.shape


(24029, 104)

In [5]:
#print("Distribution")
#print()

#for league in leagues:
#    subl = df['Div'] == league
#    dfl = df[subl]
#    print(league)
#    print(dfl["FTR"].value_counts())
#    print ()
    #print(df[df['Season']==league]
    #print(df[filt]["FTR"].value_counts())
    #print()

    

In [6]:
df['label'] = pd.factorize(df['FTR'])[0]
#df['HomeTeam'] = pd.factorize(df['HomeTeam'])[0]
#df['AwayTeam'] = pd.factorize(df['AwayTeam'])[0]
df['Referee'] = pd.factorize(df['Referee'])[0]
df['Div'] = pd.factorize(df['Div'])[0]
# 0 = premier league
df['Season'] = pd.factorize(df['Season'])[0]


In [7]:
#df['label']

In [8]:
req_columns =[ #team indep 19 features 
                'Season', 'Div',#'HomeTeam', 'AwayTeam',
    'Referee',
              
              #Bookmakers x 6 = 18 features
              'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 
              'LBH', 'LBD', 'LBA', 'WHH', 'WHD', 'WHA',  
              'VCH', 'VCD', 'VCA', 
              

              
              # Team dependent feat
              # Away Team
              'C_AC', 'C_AF', 'C_AR', 'C_AS', 'C_AST', 'C_AY', 'C_FTAG', "spi_A",'C_FTAP','F_AT',
              
              # Home Team
              'C_HC', 'C_HF', 'C_HR', 'C_HS', 'C_HST', 'C_HY', 'C_FTHG', "spi",'C_FTHP', 'F_HT',
              
              # Current match --> 'FTHP', 'FTAP',
              
            
            # "spi_A","spi",
               'label']
     

df = df[req_columns]
print(req_columns)
print(df.columns.unique)

['Season', 'Div', 'Referee', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'C_AC', 'C_AF', 'C_AR', 'C_AS', 'C_AST', 'C_AY', 'C_FTAG', 'spi_A', 'C_FTAP', 'F_AT', 'C_HC', 'C_HF', 'C_HR', 'C_HS', 'C_HST', 'C_HY', 'C_FTHG', 'spi', 'C_FTHP', 'F_HT', 'label']
<bound method Index.unique of Index(['Season', 'Div', 'Referee', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD',
       'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'WHH', 'WHD', 'WHA',
       'VCH', 'VCD', 'VCA', 'C_AC', 'C_AF', 'C_AR', 'C_AS', 'C_AST', 'C_AY',
       'C_FTAG', 'spi_A', 'C_FTAP', 'F_AT', 'C_HC', 'C_HF', 'C_HR', 'C_HS',
       'C_HST', 'C_HY', 'C_FTHG', 'spi', 'C_FTHP', 'F_HT', 'label'],
      dtype='object')>


In [9]:
df.shape

(24029, 42)

# Dd to Np & Fill NaN 

In [10]:
df.fillna(0, inplace=True) # data = np.array(df)
data = np.array(df)

In [11]:

X = data[:,:-1] / (data[:,:-1].max(axis=0))
#data = np.nan_to_num(data)
data.shape

(24029, 42)

# Train Test Split

In [12]:
X = data[:,:-1]
y = data[:,-1]
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42, shuffle=True) 
X_train, X_val, y_train, y_val = train_test_split( X_train, y_train, test_size=0.25, random_state=42,shuffle=True)

# In this way, train, val, test set will be 60%, 20%, 20% of the dataset respectively.

x_len = X_train.shape[1]

tot = len(X)

(24029, 42)

# Multilayer Perceptron

In [14]:
# MLP Model
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.models import load_model
from keras import optimizers

# Initialising the ANN
model = Sequential()

# Adding the input layer and the first hidden layer
model.add(Dense(75, activation = 'relu', input_dim = x_len))


# Adding the second hidden layer
#model.add(Dense(units = 10, activation = 'relu'))
#model.add(Dense(units = 10, activation = 'relu'))


# Adding the output layer
#model.add(Dense(units = 3))
model.add(Dense(units = 1, activation='relu'))
#model.add(Dense(units = 1))



sgd = optimizers.SGD(lr=0.01)  

model.compile(optimizer = sgd, loss = 'mean_squared_error', metrics=['accuracy'])
# layers (6 tot 15 trial )


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [15]:
fitted = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), batch_size=32)

Instructions for updating:
Use tf.cast instead.
Train on 14417 samples, validate on 4806 samples
Epoch 1/50
14417/14417 [==============================] - 1s 45us/step - loss: 1.9193 - acc: 0.4360 - val_loss: 1.3689 - val_acc: 0.4395
Epoch 2/50
14417/14417 [==============================] - 1s 45us/step - loss: 1.3604 - acc: 0.4366 - val_loss: 1.3689 - val_acc: 0.4395
Epoch 3/50
14417/14417 [==============================] - 1s 38us/step - loss: 1.3604 - acc: 0.4366 - val_loss: 1.3689 - val_acc: 0.4395
Epoch 4/50
14417/14417 [==============================] - 1s 38us/step - loss: 1.3604 - acc: 0.4366 - val_loss: 1.3689 - val_acc: 0.4395
Epoch 5/50
14417/14417 [==============================] - 1s 40us/step - loss: 1.3604 - acc: 0.4366 - val_loss: 1.3689 - val_acc: 0.4395
Epoch 6/50
14417/14417 [==============================] - 1s 40us/step - loss: 1.3604 - acc: 0.4366 - val_loss: 1.3689 - val_acc: 0.4395
Epoch 7/50
14417/14417 [==============================] - 1s 58us/step - loss: 1.

In [19]:
model.save('model_mlp1.h5')  # creates a HDF5 file 'my_model.h5'


# Statistics

In [ ]:
scores = model.evaluate(X_test, y_test)
loss = scores[0]
accuracy = scores[1]


print()
print("Loss = \t\t{}".format(loss))
print("Accuracy = \t{}".format(accuracy))
print("Random \t= \t0.33")
print("On Distr = \t{}".format(df.label.value_counts()[0]/sum(df.label.value_counts())*100))


In [ ]:
history = fitted 
   
print(history.history.keys())  
   
plt.figure(1)  
   
# summarize history for accuracy  
   
plt.subplot(211)  
plt.plot(history.history['acc'],color ='black',linestyle='dashed')  
plt.plot(history.history['val_acc'],color ='grey',linestyle='solid')  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'val'], loc='upper left')  
#plt.savefig('model-6-1.png')
   


In [ ]:
 # summarize history for loss  

plt.subplot(212)  

plt.plot(history.history['loss'],color ='black',linestyle='dashed')  
plt.plot(history.history['val_loss'],color ='grey',linestyle='solid')  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'val'], loc='upper left')  
#plt.show()  
#plt.savefig('model-6-2.png')